# IMPORTS

In [22]:
import yaml
import firebase_admin
import plotly
import plotly.plotly as py
import numpy as np
import re

import pandas as pd

import geocoder


import gspread
import httplib2
import os, io
import argparse
import auth


from firebase_admin import credentials
from firebase_admin import firestore


from httplib2 import Http
from oauth2client import file,client, tools
from googleapiclient import discovery
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from oauth2client.service_account import ServiceAccountCredentials



from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# CONNECT TO FIREBASE

In [2]:
filename='/Users/5danalfaro/OAN/credentials/firebase_parameters.json'

In [3]:
# Use a service account
cred = credentials.Certificate(filename)
firebase_admin.initialize_app(cred)

db = firestore.client()

## TEST

In [4]:
col_query = db.collection('Pompes_2019').where(u'Arrondissement', u'==', u'BIRO')

In [5]:
col= db.collection('Locations')

In [12]:
doc = col.document('Benin')

In [13]:
get_doc=doc.get()

In [14]:
print(u'{} => {}'.format(get_doc.id,get_doc.to_dict()))

Benin => {'population': 11180000, 'id': 'Benin', 'coordinates': {'longitud': '2.315834', 'latitud': '9.30769'}}


# ABRIR LAS BBDD

## OPEN GOOGLE SHEETS

## AUTHENTIFICATION

In [15]:
SCOPES = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive',
         'https://www.googleapis.com/auth/drive.file',
         'https://www.googleapis.com/auth/spreadsheets.currentonly',
         'https://www.googleapis.com/auth/spreadsheets',
         'https://www.googleapis.com/auth/presentations',
         'https://www.googleapis.com/auth/drive.scripts',
         'https://www.googleapis.com/auth/drive.appdata',
         'https://www.googleapis.com/auth/drive.metadata'
        ]
store = file.Storage('/Users/5danalfaro/OAN/credentials/storage.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('/Users/5danalfaro/OAN/credentials/client_id.json', SCOPES)
    creds = tools.run_flow(flow, store)
drive_service = discovery.build('drive', 'v3', http=creds.authorize(Http()))
sheet_service = discovery.build('sheets', 'v4', http=creds.authorize(Http()))

## GOOGLE SHETS IDS

In [16]:
spreadsheet_ids = yaml.load( open("/Users/5danalfaro/OAN/credentials/sheets_parameters.yaml", "r" ),Loader=yaml.FullLoader)
#bombas de 2018, ultima version
ID1=str(spreadsheet_ids['ID1'])
#bombas de 2016, ultima version
ID2=str(spreadsheet_ids['ID2'])
#bombas de 2009, ultima version
ID3=str(spreadsheet_ids['ID3'])
#bombas final, datos de 2016 y 2009
ID4=str(spreadsheet_ids['ID4'])
#bombas lisette
ID5=str(spreadsheet_ids['ID5'])
#demograpique data
ID6=str(spreadsheet_ids['ID6'])

## get sheets

In [104]:
ids = [ID1,ID2,ID3,ID5]
dfs = []
for ID in ids:
  #SPREADSHEET_ID = ID
  RANGE_NAME = 'Sheet1'

  gsheet = sheet_service.spreadsheets().values().get(spreadsheetId=ID, range=RANGE_NAME).execute()

  dfs.append(pd.DataFrame(gsheet.get('values', [])[1:],columns= gsheet.get('values', [])[0:1][0]))

df_2019=dfs[0]
df_2016=dfs[1]
df_2009=dfs[2]
df_lisette = dfs[3]

In [105]:
dfs[1]["Etat de l'ouvrage"].unique()

array(['Fonctionnel', 'Panne', 'En panne', 'Tête de pompe', '2 pompes',
       'Privé', '1  Pompe', '', 'Pompe', 'Pompe non', 'Faible débit',
       '1 pompe', 'Kalale'], dtype=object)

In [106]:
[df.columns for df in dfs]
#df_2016.head(2)
#df_2009.head(2)

[Index(['Arrondissement', 'Village', 'Localite', 'Nom_Local', 'Longitude',
        'Latitude', 'xCoord', 'yCoord', 'rqtListe_Pompe.Modele_Pompe'],
       dtype='object'),
 Index(['N°', 'Arrondissement', 'Village', 'Localité', 'Hameau',
        'Etat de l'ouvrage', 'Redevance', 'Année de réalisation', 'Marque'],
       dtype='object'),
 Index(['Arrondissement', 'Village', 'Localité', 'Population', 'Type d'ouvrage',
        'Etat', 'Longitude', 'Latitude', 'Domaine d'installation'],
       dtype='object'),
 Index(['Departement', 'Commune', 'Arrondissement', 'Village', 'Localite',
        'Nom_Local', 'Longitude', 'Latitude', 'xCoord', 'yCoord',
        'Type_Exhaure', 'Modele_Pompe'],
       dtype='object')]

So common data is:

* 2019 vs 2016: Arrondissement, Village, Localite, Marque

* 2019 vs 2009: Arrondissement, Village, Localite, Longitude, Latitude

* 2009 vs 2016: Arrondissement, Village, Localite, Etat

ONLY DATA:

* 2019: Nom_Local which is little identifier for more info on pumps

* 2016: Année de réalisation, , Redevace, Hameau, which would be kind of the same as Nom_Local

* 2009: Population, Type d'ouvrage

MISSING:

* 2019: Population, other villages, Année de réalisation, , Redevace, Hameau

* 2016: Population, other villages

* 2009: Année de réalisation, Redevace, Hameau

### LETS CLEAN THE DATA

#### Functions

In [107]:
#Marques des Pompes
def marques(x,name,name2=0):
    marque='NA'
  
    if x[name]:
        if bool(re.search('ergnet',x[name].lower())):
            marque='Vergnet'
        elif bool(re.search('ndia',x[name].lower())):
            marque='India'
        elif bool(re.search('afri',x[name].lower())):
            marque='Afridev'
    elif x[name2]:
        if bool(re.search('moteur',x[name2].lower())):
            marque='pompe_a_moteur'
    return marque

In [108]:
#Years
def getyear(x):
    year=None
    if x['Année de réalisation']:
        for s in x['Année de réalisation'].split():
            if s.isdigit():
                year=int(s)
        if year==None:
            return year
    elif year >2019:
        #print(year)
        year=2010
        return year
    elif year > 100:
        return int(year)
    elif year >70:
        year=int('19'+str(year))
        return int(year)
    else:
        year=None
    
    return year

In [109]:
'''
Function to change latitude/longitude data from degrees,minutes and seconds into decimals
'''
def decimals_lat_lon(x,lat):
    coord=None
    if x[lat]:
        if len(x[lat])>2:
            if len(re.split('[°\'"]', x[lat])) == 4 :
        
                #gets all the parts of the coordinates into different elements of a vector
                deg, minutes, seconds, direction =  re.split('[°\'"]', x[lat])

                #had to put replace commas because of data of 2009
                coord=(float(deg) + float(minutes.replace(",","."))/60 + float(seconds.replace(",","."))/(60*60)) * (-1 if direction in ['W', 'S'] else 1)
            elif len(re.split('[°\'"]', x[lat])) > 0:
                coord=x[lat]
                print(coord)
    else:
        coord=None
        print(coord)
    return coord

2009

In [110]:
df_2009.columns

Index(['Arrondissement', 'Village', 'Localité', 'Population', 'Type d'ouvrage',
       'Etat', 'Longitude', 'Latitude', 'Domaine d'installation'],
      dtype='object')

In [111]:
df_2009['Latitude_decimals']=df_2009.apply(lambda x: decimals_lat_lon(x,'Latitude'),axis=1)
df_2009['Longitude_decimals']=df_2009.apply(lambda x: decimals_lat_lon(x,'Longitude'),axis=1)

None
None
None
None
None
None
None
None
None
None


#### Begin with 2019

In [37]:
#Create decimals from degrees latitude
df_2019['Latitude_decimals']=df_2019.apply(lambda x: decimals_lat_lon(x,'Latitude'),axis=1)

None
None


In [38]:
df_2019['Longitude_decimals']=df_2019.apply(lambda x: decimals_lat_lon(x,'Longitude'),axis=1)

None
None


In [39]:
df_2019_filtered = df_2019[['Arrondissement', 'Village','Localite', 'Longitude_decimals',
        'Latitude_decimals', 'rqtListe_Pompe.Modele_Pompe']]
df_2019_filtered.columns = ['Arrondissement', 'Village','Localite', 'Longitude',
        'Latitude', 'Marque']

In [42]:
df_2019_filtered['Marque']=df_2019_filtered.apply(lambda x: marques(x,"Marque",),axis=1)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [44]:
l_2019_local = df_2019_filtered['Localite'].sort_values().tolist()

index = []
j=1
for i,local in enumerate(l_2019_local):
    if i == 0:
        j=1
        word = local+' '+str(j)
        word=word.replace(' ','_')
        index.append(word)
        continue
    if l_2019_local[i-1] == local:
        j+=1
        word = local+' '+str(j)
        word=word.replace(' ','_')
        index.append(word)
    else:
        j=1
        word = local+' '+str(j)
        word=word.replace(' ','_')
        index.append(word)

In [45]:
df_2019_filtered=df_2019_filtered.sort_values('Localite')

In [46]:
df_2019_filtered['index'] = index

In [47]:
df_2019_filtered=df_2019_filtered.set_index('index')

In [50]:
df_2019_filtered=df_2019_filtered.reset_index()

In [0]:
#convert to dictionary
dict_2019 = df_2019_filtered.to_dict('index')
dict_2019 = [{k: v} for k, v in dict_2019.items()]

## Process data of Lisette

In [51]:
df_lisette.head()

,Departement,Commune,Arrondissement,Village,Localite,Nom_Local,Longitude,Latitude,xCoord,yCoord,Type_Exhaure,Modele_Pompe
0,BORGOU,NIKKI,BIRO,BIRO,BIRO NOUKOUROU,,"02°56'50.96""E","09°54'18.50""N",2.947489,9.905139,PMH,VERGNET 4C
1,BORGOU,NIKKI,BIRO,BIRO,GOUTTO,,"02°56'47.38""E","09°54'59.99""N",2.946494,9.916664,PMH,VERGNET 4D
2,BORGOU,NIKKI,BIRO,BIRO,SARAVONDO,,"02°58'29.33""E","09°54'13.30""N",2.974814,9.903694,PMH,VERGNET 4D
3,BORGOU,NIKKI,BIRO,BIRO,MADINA,,"02°56'44.34""E","09°54'02.71""N",2.94565,9.900753,PMH,VERGNET 4C
4,BORGOU,NIKKI,BIRO,BIRO,MADINA,,"02°56'46.83""E","09°54'01.55""N",2.946342,9.900431,PMH,VERGNET 4C


In [0]:
#Create decimals from degrees latitude
df_lisette['Latitude_decimals']=df_lisette.apply(lambda x: decimals_lat_lon(x,'Latitude'),axis=1)

In [0]:
df_lisette['Longitude_decimals']=df_lisette.apply(lambda x: decimals_lat_lon(x,'Longitude'),axis=1)

In [0]:
df_lisette['Marque']=df_lisette.apply(lambda x: marques(x,"Modele_Pompe","Type_Exhaure"),axis=1)

In [0]:
df_lisette.groupby("Marque",axis=0).count()

,Departement,Commune,Arrondissement,Village,Localite,Nom_Local,Longitude,Latitude,xCoord,yCoord,Type_Exhaure,Modele_Pompe,Latitude_decimals,Longitude_decimals
Marque,,,,,,,,,,,,,,
Afridev,33,33,33,33,33,33,33,33,33,33,33,33,33,33
India,95,95,95,95,95,95,95,95,95,95,95,95,95,95
NA,4,4,4,4,4,4,4,4,4,4,3,0,4,4
Vergnet,253,253,253,253,253,253,253,253,253,253,253,253,253,253
pompe_a_moteur,14,14,14,14,14,14,14,14,14,14,14,0,14,14


###Analysis to check

###How many villages have pompes

In [0]:
len(df_lisette.groupby("Localite",axis=0).count())

277

In [0]:
len(df_2009.groupby("Localité",axis=0).count())

326

#STATES/COUNTY/DISTRICT


### Demographics

get data from INSAE BENIN link: https://www.insae-bj.org/statistiques/statistiques-demographiques#nos-12-departements -> placed it manually into a google sheet

then add the coordinates by Geonames API

In [0]:
#get data of RPGH4
RANGE_NAME = 'Sheet1'

gsheet = sheet_service.spreadsheets().values().get(spreadsheetId=ID6, range=RANGE_NAME).execute()

df_rpgh4 = pd.DataFrame(gsheet.get('values', [])[1:],columns= gsheet.get('values', [])[0:1][0])

##State DB

In [0]:
df_dep_base = df_rpgh4[df_rpgh4['Divison administrative'].str[0:4] == 'DEP:'].copy()

In [0]:
df_dep_base=df_dep_base.reset_index()

In [0]:
df_dep_base['Departement'] = df_dep_base.apply(lambda x: x['Divison administrative'][5:], axis=1)

In [0]:
df_dep=df_dep_base.sort_values('Departement')

In [0]:
df_dep = df_dep[['Departement','Total','Masculin','Feminin']]

### get localisation from geocoder

In [0]:
def get_coor(x):
  g=geocoder.geonames(x['Departement']+' Benin',key='5danalfaro')
  return [g.lat,g.lng]

In [0]:
df_dep['Coordinates'] = df_dep.apply(lambda x: get_coor(x),axis=1)

In [0]:
df_dep['Latitude'] = df_dep.apply(lambda x: x['Coordinates'][0],axis=1)

In [0]:
df_dep['Longitude'] = df_dep.apply(lambda x: x['Coordinates'][1],axis=1)

In [0]:
df_dep=df_dep[['Departement','Latitude','Longitude','Total']]

In [0]:
df_dep.columns = ['Name','Latitude','Longitude','Population']

Check if everything is okay, exported to csv and uploaded to google mymaps

In [0]:
df_dep.to_csv('dep.csv')

https://drive.google.com/open?id=1yJeiKBrO8j9GE10wYyRmjnQqxufG-YT0&usp=sharing all alright

## COUNTY DB

In [0]:
df_county_base = df_rpgh4[df_rpgh4['Divison administrative'].str[0:4] == 'COM:'].copy()

In [0]:
df_county_base=df_county_base.reset_index()

In [0]:
def getdep(x):
  l_dep_in = df_dep_base['index'].to_list()
  row = x['index']
  for i,index in enumerate(l_dep_in):
    if i+1< len(l_dep_in):
      if ((index < row) and (row<l_dep_in[i+1])):
        return df_dep_base[df_dep_base['index']==index]['Departement'].values[0]
    else:
      if(index < row):
        return df_dep_base[df_dep_base['index']==index]['Departement'].values[0]

In [0]:
df_county_base['Departement'] = df_county_base.apply(lambda x: getdep(x), axis=1)

In [0]:
df_county_base['Name'] = df_county_base.apply(lambda x: x['Divison administrative'][5:], axis=1)

In [0]:
df_county=df_county_base.sort_values(by=['Departement','Name'])

In [0]:
df_county = df_county[['Name','Departement','Total']]

### get localisation from geocoder

In [0]:
def get_coor_county(x):
  g=geocoder.geonames(x['Name']+' '+x['Departement']+' Benin',key='5danalfaro')
  return [g.lat,g.lng]

In [0]:
df_county['Coordinates'] = df_county.apply(lambda x: get_coor_county(x),axis=1)

In [0]:
df_county['Latitude'] = df_county.apply(lambda x: x['Coordinates'][0],axis=1)

In [0]:
df_county['Longitude'] = df_county.apply(lambda x: x['Coordinates'][1],axis=1)

In [0]:
df_county=df_county[['Name','Departement','Latitude','Longitude','Total']]

In [0]:
df_county.columns = ['Name','State','Latitude','Longitude','Population']

Check if everything is okay, exported to csv and uploaded to google mymaps

In [0]:
df_county.to_csv('dis.csv')

https://drive.google.com/open?id=1yJeiKBrO8j9GE10wYyRmjnQqxufG-YT0&usp=sharing seems all right

## DISTRICT DB

In [0]:
df_dis = df_rpgh4[df_rpgh4['Divison administrative'].str[0:7] == 'ARROND:'].copy()

In [0]:
df_dis=df_dis.reset_index()

In [0]:
def getcounty(x):
  l_county_in = df_county_base['index'].to_list()
  row = x['index']
  for i,index in enumerate(l_county_in):
    if i+1< len(l_county_in):
      if ((index < row) and (row<l_county_in[i+1])):
        return df_county_base[df_county_base['index']==index]['Name'].values[0]
    else:
      if(index < row):
        return df_county_base[df_county_base['index']==index]['Name'].values[0]

In [0]:
df_dis['Commune'] = df_dis.apply(lambda x: getcounty(x), axis=1)

In [0]:
df_dis['Departement'] = df_dis.apply(lambda x: getdep(x), axis=1)

In [0]:
df_dis['Name'] = df_dis.apply(lambda x: x['Divison administrative'][8:], axis=1)

In [0]:
df_dis=df_dis.sort_values(by=['Departement','Commune','Name'])

In [0]:
df_dis = df_dis[['Name','Departement','Commune','Total']]

### get localisation from geocoder

In [0]:
def get_coor_dis(x):
  g=geocoder.geonames(x['Name']+' '+x['Commune']+' '+x['Departement']+' Benin',key='5danalfaro')
  return [g.lat,g.lng]

In [0]:
df_dis['Coordinates'] = df_dis.apply(lambda x: get_coor_dis(x),axis=1)

In [0]:
df_dis['Latitude'] = df_dis.apply(lambda x: x['Coordinates'][0],axis=1)

In [0]:
df_dis['Longitude'] = df_dis.apply(lambda x: x['Coordinates'][1],axis=1)

In [0]:
df_dis=df_dis[['Name','Departement','Commune','Latitude','Longitude','Total']]

In [0]:
df_dis.columns = ['Name','State','County','Latitude','Longitude','Population']

Check if everything is okay, exported to csv and uploaded to google mymaps

In [0]:
df_dis.to_csv('dis.csv')

https://drive.google.com/open?id=1yJeiKBrO8j9GE10wYyRmjnQqxufG-YT0&usp=sharing some mistakes in locating the districts

### UPLOAD TO FIREBASE

In [0]:
#STATE
#Convert states df to a dictionary
states = df_dep.to_dict('index')
states_dict = [{k: v} for k, v in states.items()] 
states_l_dict = []
for i,sta in enumerate(states_dict):
  id_state=list(sta.keys())[0]
  state=sta[id_state]['Name']
  new_data={'Name':state,
            'Coordinates':{
                'Longitude':sta[id_state]['Longitude'],
                'Latitude':sta[id_state]['Latitude']},
            'Population':sta[id_state]['Population']}
  states_l_dict.append(new_data)
  
  doc_state = db.collection(u'Locations').document(u'Benin').collection(u'States').add(new_data)[1]
  if i ==1:
    break

In [0]:
doc_state.update({'id':doc_state.id})

update_time {
  seconds: 1568999130
  nanos: 556021000
}

In [0]:
doc_state.get().add({'id':doc_state.id})

AttributeError: ignored

In [0]:
new_data

{'Coordinates': {'Latitude': '10.30416', 'Longitude': '1.37962'},
 'Name': 'ATACORA',
 'Population': '772262',
 'id': 'hey'}

In [0]:
#STATE
#Convert states df to a dictionary
states = df_dep.to_dict('index')
states_dict = [{k: v} for k, v in states.items()] 
states_l_dict = []
districts_l_dict = []
countys_l_dict = []
for sta in states_dict:
  id_state=list(sta.keys())[0]
  state=sta[id_state]['Name']
  new_data={'name':state,
            'coordinates':{
                'longitude':sta[id_state]['Longitude'],
                'latitude':sta[id_state]['Latitude']},
            'population':sta[id_state]['Population']}
  
  doc_state = db.collection(u'Locations').document(u'Benin').collection(u'States').add(new_data)[1]
  state_id = doc_state.id
  doc_state.update({'id':state_id})
  new_data['id']=state_id
  states_l_dict.append(new_data)

  
  #COUNTY
  #get countys of that state
  df_county_state = df_county[df_county['State']==state]
  #Convert countys df to a dictionary
  countys = df_county_state.to_dict('index')
  countys_dict = [{k: v} for k, v in countys.items()] 
  for coun in countys_dict:
    id_county=list(coun.keys())[0]
    county=coun[id_county]['Name']
    new_data={'name':county,
              'coordinates':{
                  'longitude':coun[id_county]['Longitude'],
                  'latitude':coun[id_county]['Latitude']},
              'population':coun[id_county]['Population'],
              'state':state,
              'state_id':state_id}
    doc_county = db.collection(u'Locations').document(u'Benin').collection(u'County').add(new_data)[1]
    county_id=doc_county.id
    doc_county.update({'id':county_id})
    new_data['id']=county_id
    countys_l_dict.append(new_data)
    
    #DISTRICT
    #get districts of that county
    df_districts_county = df_dis[df_dis['County']==county]
    #Convert district df to a dictionary
    districts = df_districts_county.to_dict('index')
    districts_dict = [{k: v} for k, v in districts.items()] 
    for dis in districts_dict:
      id_districts=list(dis.keys())[0]
      district = dis[id_districts]['Name']
      new_data={'name':district,
                'coordinates':{
                    'longitude':dis[id_districts]['Longitude'],
                    'latitude':dis[id_districts]['Latitude']},
                'population':dis[id_districts]['Population'],
               'state':state,
               'state_id':state_id,
               'county':county,
               'county_id':county_id}
      doc_district = db.collection(u'Locations').document(u'Benin').collection(u'District').add(new_data)[1]
      district_id=doc_district.id
      doc_county.update({'id':district_id})
      new_data['id']=district_id
      districts_l_dict.append(new_data)
  

## VILLAGES

Only Nikki - get villages from work done by FICAI: https://www.google.com/maps/d/edit?mid=1te53KEJYUJ2ulJLsGpa9S15suY1lNPGQ&ll=10.108170603054887%2C2.9732788000000028&z=8

In [0]:
import xml.sax, xml.sax.handler
import re
from zipfile import ZipFile

In [0]:
filename = 'Poblacion.kmz'

kmz = ZipFile(filename, 'r')
kml = kmz.open('doc.kml', 'r')

In [0]:

class PlacemarkHandler(xml.sax.handler.ContentHandler):
    def __init__(self):
        self.inName = False # handle XML parser events
        self.inPlacemark = False
        self.mapping = {} 
        self.buffer = ""
        self.name_tag = ""
        
    def startElement(self, name, attributes):
        if name == "Placemark": # on start Placemark tag
            self.inPlacemark = True
            self.buffer = "" 
        if self.inPlacemark:
            if name == "name": # on start title tag
                self.inName = True # save name text to follow
            
    def characters(self, data):
        if self.inPlacemark: # on text within tag
            self.buffer += data # save text if in title
            
    def endElement(self, name):
        self.buffer = self.buffer.strip('\n\t')
        
        if name == "Placemark":
            self.inPlacemark = False
            self.name_tag = "" #clear current name
        
        elif name == "name" and self.inPlacemark:
            self.inName = False # on end title tag            
            self.name_tag = self.buffer.strip()
            self.mapping[self.name_tag] = {}
        elif self.inPlacemark:
            if name in self.mapping[self.name_tag]:
                self.mapping[self.name_tag][name] += self.buffer
            else:
                self.mapping[self.name_tag][name] = self.buffer
        self.buffer = ""

In [0]:
parser = xml.sax.make_parser()
handler = PlacemarkHandler()
parser.setContentHandler(handler)
parser.parse(kml)
kmz.close()

In [0]:
def build_table(mapping):
    sep = ','
        
    output = []
    shapes = ''
    for key in mapping:
        coord_str = mapping[key]['coordinates']
        coords = re.findall('\d.*,.*\d', coord_str )
        coord = coords[0].replace(',0',"")
        coord = coord.split(',')
        name = key.split(' - ')
        if len(name)==1:
          continue
        else:
          output.append({'Name':name[1],'Arrondissement':name[0],'Longitude':coord[0],'Latitude':coord[1]})
    return output

In [0]:
outstr = build_table(handler.mapping)

In [0]:
df_villages = pd.DataFrame(outstr)

In [0]:
df_villages = df_villages[['Name','Arrondissement','Longitude','Latitude']]

In [0]:
df_villages['Arrondissement'] = df_villages.apply(lambda x: 'GNONKOUROKALI' 
                                                  if bool(re.search('gnonkou',x['Arrondissement'].lower()))
                                                  else str(x['Arrondissement']).upper(), 
                                                  axis=1)

In [0]:
def cosa(x):
  arron = x['Arrondissement']
  village = x['Name']
  df_arron = df_2009[df_2009['Arrondissement']==arron]
  
  choices = df_arron['Localité'].to_list()
  
  results = process.extract(village, choices, limit=5)
  
  if len(results)>0:
  
    result = results[0][0]
    value = results[0][1]
    if value <85:
      print(village)
      print(results)
      print('FINI\n\n')

    population = df_2009[df_2009['Localité']==result].values[0]
    
  else:
    population = None
    print(village)
    print('CACA\n')
    
  return population
    

    
    

In [0]:
df_villages['Population'] = df_villages.apply(lambda x: cosa(x),axis=1)

Gneltoko
[('Bokon', 60), ('Gnonho', 57), ('Gnonkourokali', 56), ('Gnonkourokali', 56), ('Gnonkourokali', 56)]
FINI


Gourou Pibou
[('Dakou Gourou', 71), ('Koro', 68), ('Guinrou', 64), ('Guinrou', 64), ('Guinrou', 64)]
FINI


Weketere
[('Wèkètèrè', 67), ('Gotel centre', 50), ('Gnel Kéridji', 32), ('Gotel Peuhl', 32), ('Kongaré', 29)]
FINI


Ouromonsi Peulh
[('Ouroumon', 79), ('Ouroumon', 79), ('Ouroumon', 79), ('Bouérou', 60), ('Ouénou', 54)]
FINI


GnelKiradje
[('Gnel Siradjè', 82), ('Gnel Kéridji', 73), ('Gnel Bodédji', 55), ('Kibia', 54), ('Gnel Sanna', 48)]
FINI


Ganrou Peulh
[('Ganrou Bariba', 64), ('Ganrou Bariba', 64), ('Ganrou Bariba', 64), ('Ganrou Gando', 64), ('Gah Sonna Peuhl', 59)]
FINI


Bantere
[('Bantéré', 83), ('Bantéré', 83), ('Dantcha', 43), ('Gnannoubéré', 38), ('Gnannoubéré', 38)]
FINI


Ganchon
[('Ganson', 77), ('Ganson', 77), ('Ganson', 77), ('Gah Sonna', 62), ('Gando Pénéré', 62)]
FINI


Gbarire
[('Gbabiré', 77), ('Gbabiré', 77), ('Gbabiré Zongo', 64), ('Boa', 6

In [0]:
village = 'Gourou Pibou'
arron = df_villages[df_villages['Name']==village]['Arrondissement'].values[0]
df_arron = df_2009[df_2009['Arrondissement']==arron]

choices1 = df_2009['Village'].to_list()
choices2 = df_2009['Localité'].to_list()

results1=process.extract(village, choices1, limit=5)
results2=process.extract(village, choices2, limit=5)
print(results1)
print(results2)

[('Gourou', 90), ('Gourou', 90), ('Gourou', 90), ('Gourou', 90), ('Gourou', 90)]
[('Gourou', 90), ('Gourou', 90), ('Gourou', 90), ('Gourou', 90), ('Gourou', 90)]


In [0]:
df_arron

,Arrondissement,Village,Localité,Population,Type d'ouvrage,Etat,Longitude,Latitude,Domaine d'installation,Latitude_decimals,Longitude_decimals
57,GNONKOUROKALI,Gbari,Angaradébou Peuhl,140,/,/,/,/,/,NaN,NaN
58,GNONKOUROKALI,Gbari,Dassarou Peuhl,650,FPM,Fonctionnel,"02°55'07,8""E","10°01'31,2""N",Domaine public,10.025333,2.918833
59,GNONKOUROKALI,Gbari,Dassarou Peuhl,650,FPM,Fonctionnel,"02°55'40,1""E","10°01'52,5""N",Domaine public,10.031250,2.927806
60,GNONKOUROKALI,Gbari,Gah Gosso,450,/,/,/,/,/,NaN,NaN
61,GNONKOUROKALI,Gbari,Gbari,1132,FPM,Fonctionnel,"02°55'51,9""E","10°00'22,0""N",Domaine public,10.006111,2.931083
62,GNONKOUROKALI,Gbari,Gbari,1132,FPM,Fonctionnel,"02°55'43,7""E","10°00'24,5""N",Domaine public,10.006806,2.928806
63,GNONKOUROKALI,Gbari,Gbari,1132,PM,Abandonné,"02°55'49,2""E","10°00'15,8""N",Domaine public,10.004389,2.930333
64,GNONKOUROKALI,Gbari,Gbari,1132,PM,Non fonctionnel,"02°55'50,4""E","10°00'16,6""N",Domaine public,10.004611,2.930667
65,GNONKOUROKALI,Gbari,Gbari,1132,PM,Fonctionnel,"02°55'54,2""E","10°00'17,0""N",Domaine public,10.004722,2.931722
66,GNONKOUROKALI,Gbari,Gbari,1132,PM,Fonctionnel,"02°55'21,5""E","10°00'03,9""N",Domaine public,10.001083,2.922639


In [0]:
arron

'GNONKOUROKALI'

In [0]:
df_2009[df_2009['Localité']==results1[0][0]]

,Arrondissement,Village,Localité,Population,Type d'ouvrage,Etat,Longitude,Latitude,Domaine d'installation,Latitude_decimals,Longitude_decimals
157,NIKKI,Gourou,Gourou,5905,PM,Fonctionnel,"03°12'34,5""E","09°56'23,2""N",Domaine public,9.939778,3.209583
158,NIKKI,Gourou,Gourou,5905,PM,Fonctionnel,"03°12'33,4""E","09°56'23,0""N",Domaine public,9.939722,3.209278
159,NIKKI,Gourou,Gourou,5905,PM,Fonctionnel,"03°12'34,3""E","09°56'17,8""N",Domaine public,9.938278,3.209528
160,NIKKI,Gourou,Gourou,5905,PM,Fonctionnel,"03°12'30,8""E","09°56'16,8""N",Domaine public,9.938000,3.208556
161,NIKKI,Gourou,Gourou,5905,PT,Fonctionnel,"03°12'37,0""E","09°56'09,2""N",Palai Royal,9.935889,3.210278
162,NIKKI,Gourou,Gourou,5905,PM,Fonctionnel,"03°12'40,9""E","09°56'06,9""N",Domaine public,9.935250,3.211361
163,NIKKI,Gourou,Gourou,5905,FPM,Fonctionnel,"03°12'35,5""E","09°56'03,4""N",Mairie de Nikki,9.934278,3.209861
164,NIKKI,Gourou,Gourou,5905,PM,Fonctionnel,"03°12'28,9""E","09°56'10,0""N",Marché de Nikki,9.936111,3.208028


In [0]:
df_2009['Arrondissement'].unique()

array(['BIRO', 'GNONKOUROKALI', 'NIKKI', 'OUENOU', 'SEREKALE', 'SUYA',
       'TASSO'], dtype=object)

## Process the Data

### FUZZY TO COMPARE Localites

### TEST

In [23]:
#Example of use
choices = df_2016['Localité'].tolist()
localite = df_2019.loc[20,'Localite']
process.extract(localite, choices, limit=5)


[('Ourarou', 100),
 ('Ourarou', 100),
 ('Ourarou', 100),
 ('Ourarou Gberou kpanin', 90),
 ('Gnelkourarou', 90)]

In [24]:
#another one
word1=df_2019.loc[108,'Localite']
word2=df_2016.loc[150,'Localité']
print(word1,'\n',word2)

Kparissérou Gando 
 Kparisserou gando


In [25]:
fuzz.ratio(word1, word2)

88

In [26]:
word1=word1+'!'
fuzz.partial_ratio(word1, word2)

88

In [27]:
fuzz.ratio(word1, word2)

86

In [28]:
process.extract('Gando Gourou', df_2019['Localite'], limit=5)


[('Gando Gourou', 100, 329),
 ('Gourou', 90, 109),
 ('Gourou', 90, 110),
 ('Gourou', 90, 111),
 ('Dèh Gando', 86, 172)]

In [29]:
#last one
word1=df_2019.loc[329,'Localite']
word2=df_2016.loc[270,'Localité']
print(word1,'\n',word2)

Gando Gourou 
 Ganrou gando


In [30]:
fuzz.ratio(word1, word2)

42

In [31]:
fuzz.token_sort_ratio(word1, word2)

83

# WHAT VILLAGES DON'T HAVE FPM

In [114]:
df_2009.columns

Index(['Arrondissement', 'Village', 'Localité', 'Population', 'Type d'ouvrage',
       'Etat', 'Longitude', 'Latitude', 'Domaine d'installation',
       'Latitude_decimals', 'Longitude_decimals'],
      dtype='object')

In [150]:
df_villages_2009 = df_2009.groupby('Localité').agg(
                            {
                            "Type d'ouvrage":'unique',
                            "Population":'max',
                            "Latitude_decimals":'mean',
                            "Longitude_decimals":'mean',
                            "Arrondissement":'unique'})

In [151]:
df_villages_2009 = df_villages_2009.reset_index()

In [152]:
df_villages_2009.Arrondissement=df_villages_2009.apply(lambda x: x['Arrondissement'][0],axis=1)

In [153]:
df_villages_2009.Population=pd.to_numeric(df_villages_2009.Population, errors='coerce')

In [154]:
def count_pm_fpm(x):
    types_ouvrage=list(x["Type d'ouvrage"])
    pm=types_ouvrage.count('PM')
    fpm=types_ouvrage.count('FPM')
    pt=types_ouvrage.count('PT')
    none = types_ouvrage.count('/')
    
    if(len(types_ouvrage)!= (pm+fpm+pt+none)):
        print(types_ouvrage)
    return [pm,fpm,pt]

In [155]:
df_villages_2009[['PM','FPM','PT']]=df_villages_2009.apply(lambda x: pd.Series(count_pm_fpm(x)), axis=1)

['FPM', 'PT', 'PM', 'BF 1', 'BF 2', 'BF 3', 'BF 4', 'BF 5']
['PEA (Rampes de 4 robinets)', 'FPM']


In [156]:
df_villages_2009=df_villages_2009.drop("Type d'ouvrage",axis=1)

In [189]:
df_villages_2009['n_per/fpm'] = df_villages_2009.apply(
                                    lambda x: 10000 if x['FPM']==0 else x['Population']/x['FPM'],axis=1)

In [191]:
def status(x):
    ind=x['n_per/fpm']
    status='unknown'
    if np.isnan(ind) or isinstance(ind, str):
        return status
    elif ind<250:
        status='green'
    elif ind <500:
        status='orange'
    elif ind==10000:
        status='super_red'
    elif ind>500:
        status='red'
    return status

In [192]:
df_villages_2009['status'] = df_villages_2009.apply(lambda x: status(x),axis=1)

In [202]:
df_villages_2009.columns=['village','population','latitude','longitude','arrondissement','pm','fpm','pt','ind','color']


In [ ]:
df_villages_2009

In [195]:
#STATE
#Convert states df to a dictionary
villages = df_villages_2009.to_dict('index')
villages_dict = [{k: v} for k, v in villages.items()] 
villages_l_dict = []
for i,vill in enumerate(villages_dict):
    id_vill=list(vill.keys())[0]
    village=vill[id_vill]['village']
    new_data={'name':village,
              'arrondissement':vill[id_vill]['arrondissement'],
              'coordinates':{
                    'longitude':vill[id_vill]['longitude'],
                    'latitude':vill[id_vill]['latitude']},
              'population':vill[id_vill]['population'],
              'pm':vill[id_vill]['pm'],
              'fpm':vill[id_vill]['fpm'],
              'pt':vill[id_vill]['pt'],
              'ind':vill[id_vill]['ind'],
              'color':vill[id_vill]['color']

             
             }
    villages_l_dict.append(new_data)

    db.collection(u'mapa_temp').add(new_data)[1]

Localité
Arrondissement status             
BIRO           green             2
               orange            6
               red               7
               super_red        16
GNONKOUROKALI  green             2
               orange            8
               red               6
               super_red        14
               unknown           3
NIKKI          green            15
               orange           15
               red              17
               super_red        53
OUENOU         green             4
               orange            6
               red               7
               super_red        40
               unknown           1
SEREKALE       green             6
               orange            8
               red               6
               super_red        14
               unknown           6
SUYA           green             1
               orange            3
               red               5
               super_red         3
               unknown           2
TASSO          green             5
               orange            9
               red               8
               super_red        28